In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls /content/drive/My\ Drive/




 1.PNG
 1.PNG.gdoc
'Atieh Gharib-CV (1).pdf'
'Atieh Gharib-CV (2).pdf'
'Atieh Gharib-CV.pdf'
 bert_summarization
 books
'Colab Notebooks'
'Copy of Glenn%20Neely%20-%20Mastering%20Elliott%20Wave.pdf'
'Copy of HSIC.zip'
 data_structures1-real1.rar
 data_structures1-real.rar
 data_structures.rar
'Digital Dollar Sign PowerPoint Templates.pptx'
 docker_images
'elliott pres2.pptx'
 elliott-wave-pattern2.rar
 EX07
 EX14.ipynb
 Forex_codes
'Gharib Atieh-CV.pdf'
 GISPython.pdf
 Glenn%20Neely%20-%20Mastering%20Elliott%20Wave.pdf
 HSIC
 hsicfeaturegumbelsparsemax_autos.pkl
 HSICNet
 HSIC.zip
 Inverse_Problem_of_Ultrasound_Beamforming_With_Denoising-Based_Regularized_Solutions.pdf
 learning
'Letter of employment, Atieh Gharib, special consulant 1.1.2025 - DTU Biosustain.pdf'
'New Doc 2020-03-12 16.06.20.gdoc'
'New Doc 2020-03-12 16.06.20.pdf'
 ONSPEED-DE-UDP.ovpn
 ONSPEED-FR-UDP.ovpn
 Openvpn
'Proposal Neth Calls'
'published data'
'published data.zip'
 sherkat
'Sworn declaration AR6.pdf'
 Untitled

In [7]:
%cd /content/drive/My\ Drive/HSIC/
!ls  # Lists files in the current folder


/content/drive/My Drive/HSIC
consistency_scores.csv	feature_importances_new.pkl  __pycache__       selected_features_new.pkl
data			HSICNet			     real_datasets.py  trained_models


In [8]:
!pip install sparsemax
!pip install ucimlrepo


In [17]:
import torch
import os
from real_datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min, pairwise_distances
import pickle
import numpy as np
from openpyxl import Workbook
from real_datasets import load_dataset
from HSICNet.HSICFeatureNet import *
from HSICNet.HSICNet import *
from HSICNet.util import *
import gc
import torch



In [33]:
def load_models(dataset_name, X_tensor_tbx):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print(f"Applying HSICFeatureNetGumbelSparsemax on dataset: {dataset_name}")
    # with open('trained_models/hsicfeaturegumbelsparsemax_autos.pkl', 'rb') as f:
    with open('trained_models/hsicfeaturegumbelsparsemax_bike.pkl', 'rb') as f:
        # featuregumbelsparsemax_model = torch.load(f, map_location=torch.device('cpu'))
        # featuregumbelsparsemax_model = torch.load(f, map_location=device)
        featuregumbelsparsemax_model = pickle.load(f)
    featuregumbelsparsemax_model.eval()
    with torch.no_grad():  # Disable gradient computation
        hsic_fNET_gsp_weights, _ ,_ , _= featuregumbelsparsemax_model(X_tensor_tbx)
    HSICFGSP_selected_features = (hsic_fNET_gsp_weights > 1e-3).to(torch.int32)


    print(f"Applying HSICGumbelSparsemax on dataset: {dataset_name}")
    # with open('trained_models/hsicnetgumbelsparsemax_autos.pkl', 'rb') as f:
    with open('trained_models/hsicnetgumbelsparsemax_bike.pkl', 'rb') as f:
        gumbelsparsemax_model = pickle.load(f)
    gumbelsparsemax_model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation
            hsic_gsp_weights, _ ,_ = gumbelsparsemax_model(X_tensor_tbx)
    gumbelsparsemax_selected_features = (hsic_gsp_weights > 1e-3).to(torch.int32)

    # model_filename = f"trained_models/hsicnetgumbelsparsemax_{dataset_name}.pkl"

    feature_importances_new = [hsic_gsp_weights, hsic_fNET_gsp_weights]
    selected_features_new = [gumbelsparsemax_selected_features, HSICFGSP_selected_features]
    # Save feature_importances to a .pkl file
    with open('feature_importances_new.pkl', 'wb') as f:
            pickle.dump(feature_importances_new, f)

    with open('selected_features_new.pkl', 'wb') as f:
        pickle.dump(selected_features_new, f)

In [10]:
import os
print(os.getcwd())
!ls

/content/drive/My Drive/HSIC
consistency_scores.csv	feature_importances_new.pkl  __pycache__       selected_features_new.pkl
data			HSICNet			     real_datasets.py  trained_models


In [31]:
def consistency_across_methods(feature_matrix, feature_importances_list, method_names, n_clusters=5):
    """
    Evaluate the consistency of feature importance values across similar (clustered) instances
    for multiple attribution methods.

    Parameters:
        feature_matrix (numpy.ndarray): Test data feature matrix of shape [n_instances, n_features].
                                         This represents input instances in feature space.
        feature_importances_list (list of numpy.ndarray): A list of feature importance matrices.
                                                          Each matrix corresponds to one method and has shape [n_instances, n_features].
        method_names (list of str): Names of the methods corresponding to the feature importance matrices.
        n_clusters (int): Number of clusters to group similar instances in feature space.

    Returns:
        consistency_scores (dict): A dictionary with method names as keys and their consistency scores
                                    (list of float scores for each cluster) as values.
    """
    feature_matrix = np.array(feature_matrix)  # Ensure feature_matrix is a NumPy array
    consistency_scores = {}

    for method_idx, (method_name, feature_importances) in enumerate(zip(method_names, feature_importances_list)):
        print(f"Evaluating consistency for method: {method_name}...")
        # feature_importances = np.array(feature_importances)  # Ensure importance matrix is a NumPy array
        feature_importances = feature_importances.cpu().numpy()
        # Perform clustering using K-Means
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_labels = kmeans.fit_predict(feature_matrix)

        # Measure consistency within each cluster
        cluster_consistency = []

        for cluster_idx in range(n_clusters):
            # Indices of instances in the current cluster
            cluster_indices = np.where(cluster_labels == cluster_idx)[0]
            cluster_features = feature_matrix[cluster_indices]  # Features of instances in the cluster
            cluster_importances = feature_importances[cluster_indices]  # Importances of instances in the cluster

            # If the cluster has just one element, skip it
            if len(cluster_indices) <= 1:
                cluster_consistency.append(0)  # Small cluster, by definition consistent
                continue

            # 1. Compute pairwise distances between instances in the cluster (feature space)
            pairwise_feature_distances = pairwise_distances(cluster_features, metric='euclidean')

            # 2. Compute pairwise distances between feature importance vectors (importance space)
            pairwise_importance_distances = pairwise_distances(cluster_importances, metric='euclidean')

            # 3. Normalize importance distances by feature space distances (consistency measure)
            # Avoid division by zero in distances
            normalized_distances = np.divide(
                pairwise_importance_distances,
                pairwise_feature_distances + 1e-8  # Small epsilon to prevent division by zero
            )

            # Use mean normalized pairwise distance as the consistency score for this cluster
            cluster_consistency_score = np.mean(normalized_distances)
            cluster_consistency.append(cluster_consistency_score)

        # Store the consistency results for the current method
        consistency_scores[method_name] = cluster_consistency

    return consistency_scores

In [32]:
def save_consistency_scores_to_csv(consistency_scores, dataset_name, filename="consistency_scores.csv"):
    """
    Save consistency scores to a CSV file, including the dataset name.

    Parameters:
        consistency_scores (dict): A dictionary where keys are method names and values are lists of
                                    consistency scores for each cluster.
        dataset_name (str): The name of the dataset.
        filename (str): The name of the CSV file to save the results in.
    """
    # Prepare data for CSV
    data = []
    for method_name, scores in consistency_scores.items():
        for cluster_idx, score in enumerate(scores):
            # Create a row for each (method, cluster, score)
            data.append({
                "Dataset": dataset_name,
                "Method": method_name,
                "Cluster": cluster_idx + 1,
                "Consistency Score": score
            })

    # Convert to pandas DataFrame
    df = pd.DataFrame(data)

    # Save to a CSV file
    df.to_csv(filename, index=False)

    print(f"Consistency scores saved to {filename}.")



In [34]:
if __name__ == "__main__":

    print(torch.cuda.is_available())
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    method_names = [
            'Hsic_GumbelSparsemax',
            'HSICFeatureNet_GumbelSparsemax'

        ]
    # dataset_names = ["breast_cancer", "sonar", "nomao", "steel", "breast_cancer_wisconsin", "skillcraft", "ionosphere", "sml", "pol", \
                        #  'parkinson', 'keggdirected', "pumadyn32nm", "crime", "gas", 'autos', 'bike', 'keggundirected']
    datasets= ['bike']
        # Main running part of the script
        # for dataset_name in dataset_names:
    for data in datasets:
        print(f"\nProcessing dataset: {data}")
        X, y = load_dataset(data)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
        n, d = X_train.shape
        X_tensor= torch.tensor(X_test, dtype=torch.float32).to(device=device)
        sampleNo_tbx = 200

        load_models(data, X_tensor)
        with open('feature_importances_new.pkl', 'rb') as f:
            feature_importances = pickle.load(f)

        # Plot feature importances for 10 randomly selected instances
        # plot_feature_importances_matrix(feature_importances, method_names, num_instances=10)
    ##-------------------
    #2nd experiment
        # Evaluate consistency across methods
        n_clusters = 6  # Number of clusters to group similar instances
        consistency_scores = consistency_across_methods(X_test, feature_importances, method_names, n_clusters)

        # Output consistency scores for each method
        print("\nConsistency Scores (Lower is better):")
        for method_name, scores in consistency_scores.items():
            print(f"Method: {method_name}")
            for cluster_idx, score in enumerate(scores):
                print(f"  Cluster {cluster_idx + 1}: Consistency Score = {score:.20f}")


save_consistency_scores_to_csv(consistency_scores, data, filename="consistency_scores.csv")

True

Processing dataset: bike
Applying HSICFeatureNetGumbelSparsemax on dataset: bike
Applying HSICGumbelSparsemax on dataset: bike
Evaluating consistency for method: Hsic_GumbelSparsemax...
Evaluating consistency for method: HSICFeatureNet_GumbelSparsemax...

Consistency Scores (Lower is better):
Method: Hsic_GumbelSparsemax
  Cluster 1: Consistency Score = 0.01594365786511106609
  Cluster 2: Consistency Score = 0.00815244822348616671
  Cluster 3: Consistency Score = 0.00533627678976368678
  Cluster 4: Consistency Score = 0.00357485286410437909
  Cluster 5: Consistency Score = 0.01103201116013217790
  Cluster 6: Consistency Score = 0.01079698394507865207
Method: HSICFeatureNet_GumbelSparsemax
  Cluster 1: Consistency Score = 0.00081465835897546394
  Cluster 2: Consistency Score = 0.00049837829331826958
  Cluster 3: Consistency Score = 0.00046918501895984016
  Cluster 4: Consistency Score = 0.00050685102759669099
  Cluster 5: Consistency Score = 0.00133236433203584603
  Cluster 6: Con

In [35]:
!ls


consistency_scores.csv	feature_importances_new.pkl  __pycache__       selected_features_new.pkl
data			HSICNet			     real_datasets.py  trained_models


In [36]:
from google.colab import files

# Download the CSS file
files.download('consistency_scores.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>